1. 定义数据

In [2]:
def createDataSet():
    dataset = [[0, 0, 0, 0, 'N'], 
               [0, 0, 0, 1, 'N'], 
               [1, 0, 0, 0, 'Y'], 
               [2, 1, 0, 0, 'Y'], 
               [2, 2, 1, 0, 'Y'], 
               [2, 2, 1, 1, 'N'], 
               [1, 2, 1, 1, 'Y']]
    labels = ['outlook', 'temperature', 'humidity', 'windy']
    return dataset, labels


def createDataSet2():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    return dataSet, labels

In [46]:
dataSet,labels = createDataSet2()
print(dataSet)
print(labels)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['no surfacing', 'flippers']


2. 信息熵

In [47]:
# 无论是最初未划分前的数据，还是划分后的数据，信息熵的计算都是根据最后一列计算的
# 具体来说，依赖于最后一列的统计量，统计最后一列{取值：个数}
def unique(dataSet):
    d = {}
    column = [example[-1] for example in dataSet]
    for v in column:
        if(v not in d.keys()): d[v] = 0
        d[v] += 1
    return d
print(unique(dataSet))

{'yes': 2, 'no': 3}


In [48]:
# 根据类别统计计算信息熵
from math import log
def entropy(dataSet):
    sample_num = len(dataSet)
    result = 0.0
    for k,v in unique(dataSet).items():
        prob = v / sample_num
        result -= prob * log(prob,2)
    return result
print(entropy(dataSet))

0.9709505944546686


3. 选择切分数据的最佳特征（根据特征的取值，切分数据）


In [49]:
# 根据(feature_index，value)划分数据集

def split_dataset(dataset, feature_index, value):
    sub_dataset = []
    for sample in dataset:
        if sample[feature_index] == value:
            sub_dataset.append(sample)
    return sub_dataset

# 返回分裂特征的索引
print('以第0个特征为分裂特征进行分裂数据集，')
print('分裂特征值为0的子集合：',splitDataSet(dataSet,0,0))
print('分裂特征值为1的子集合：',splitDataSet(dataSet,0,1))
print('分裂特征值为2的子集合：',splitDataSet(dataSet,0,2))

以第0个特征为分裂特征进行分裂数据集，
分裂特征值为0的子集合： [[0, 1, 'no'], [0, 1, 'no']]
分裂特征值为1的子集合： [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no']]
分裂特征值为2的子集合： []


In [59]:

# 维护一个未使用特征索引列表，
# 对于未使用的特征，计算信息增益，选择最佳特征，加入到表示决策树的嵌套字典中
# 参数1 划分后的数据集，参数2 未使用特征索引列表
def choose_best_feature(dataset,rest_features):
    
    #当前节点数据集的信息熵（可能是根，也可能是叶子节点）
    base_entropy = entropy(dataset)
    # 信息增益
    best_infogain = 0.0; best_feature_index = -1# 初始化   
    
    #对于未使用的特征，遍历计算信息增益
    for feature_index in rest_features:
        #当前列对应的特征取值列表
        feature_value_list = [sample[feature_index] for sample in dataset]
        unique_val = set(feature_value_list)
        new_entropy = 0.0
        
        #计算信息增益
        for value in unique_val:
            #按照第feature_index的value列划分数据集
            sub_dataset = split_dataset(dataset,feature_index,value)
            prob = len(sub_dataset)/float(len(dataset))
            new_entropy += prob * entropy(sub_dataset)
       
        info_gain = base_entropy - new_entropy        
        
        print('feature_index:{0},info_gain:{1}'.format(feature_index,info_gain))
        
        if(info_gain > best_infogain):
            best_infogain = info_gain
            best_feature_index = feature_index
            
        print('best_feature_index:{},best_infogain:{}'.format(best_feature_index,best_infogain))
    return best_feature_index
        
bestFeature = choose_best_feature(dataSet,[0,1])     
print(bestFeature)

feature_index:0,info_gain:0.4199730940219749
best_feature_index:0,best_infogain:0.4199730940219749
feature_index:1,info_gain:0.17095059445466854
best_feature_index:0,best_infogain:0.4199730940219749
0


In [51]:
a = 0 
b = 1
print('a={0}'.format(a))

a=0


4. 构建决策树

In [60]:
import operator
 
#多数表决
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [63]:
def build_tree(dataset,rest_labels,labels):
    class_list = [sample[-1] for sample in dataset]
    #终止条件
    # 类别完全相同则停止继续划分，返回类别
    if class_list.count(class_list[0]) == len(class_list): 
        return class_list[0]
    
    if len(rest_labels) == 0: # 遍历完所有特征时返回出现次数最多的
        return majorityCnt(class_list)
    
    print('rest_labels=>{0}'.format(rest_labels))
    ### 选择最佳特征，划分数据集
    best_feature_index = choose_best_feature(dataset,rest_labels)
    best_feature_label = labels[best_feature_index]
    myTree = {best_feature_label:{}}
    #在特征列表中，删除指定特征索引
    rest_labels.remove(best_feature_index)
    print('best_feature_index=>{0},rest_labels=>{1}'.format(best_feature_index,rest_labels))
    
    #递归调用
    featValues = [example[best_feature_index] for example in dataset]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        #递归调用的参数，切分后的数据集，剩余的特征，全部的labels只是为了构建树需要
        print('best_feature_index=>{},value=>{}'.format(best_feature_index,value))
        myTree[best_feature_label][value] = build_tree(split_dataset(dataset, best_feature_index, value),rest_labels,labels)
    return myTree 

rest_labels = list(range(0,len(labels)))
myTree= build_tree(dataSet,rest_labels,labels)

import json
print(json.dumps(myTree,indent=4))

rest_labels=>[0, 1]
feature_index:0,info_gain:0.4199730940219749
best_feature_index:0,best_infogain:0.4199730940219749
feature_index:1,info_gain:0.17095059445466854
best_feature_index:0,best_infogain:0.4199730940219749
best_feature_index=>0,rest_labels=>[1]
best_feature_index=>0,value=>0
best_feature_index=>0,value=>1
rest_labels=>[1]
feature_index:1,info_gain:0.9182958340544896
best_feature_index:1,best_infogain:0.9182958340544896
best_feature_index=>1,rest_labels=>[]
best_feature_index=>1,value=>0
best_feature_index=>1,value=>1
{
    "no surfacing": {
        "0": "no",
        "1": {
            "flippers": {
                "0": "no",
                "1": "yes"
            }
        }
    }
}


In [42]:
ag=list(range(1,10))
print(type(ag))

<class 'list'>
